In [1]:
# Anamorphic Encryption - ElGamal Scheme
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from char_int_elgamal import encode_msg_to_int, decode_msg_to_int
import secrets
import math
from itertools import zip_longest

<h1> ElGamal: Baby step - Giant step </h1>

In [20]:
# ======== HELPER FUNCTIONS ======== #


# Pseudo-random function
def F(sk_a, IV, st, q):
    prg_input = st.to_bytes(16, 'little')  # AES block size = 16 bytes
    iv_bytes = IV.to_bytes(16, 'little')
    
    aes = AES.new(sk_a, AES.MODE_CBC, iv=iv_bytes)
    encrypted = aes.encrypt(prg_input)
    
    r_prime = (int.from_bytes(encrypted, 'little') % (q - 1)) + 1
    return r_prime

def g_generator(p):
    while True:
        h = secrets.randbelow(p-3) + 2
        g = pow(h, 2, p)
        if g != 1:
            return g


# ======== KEY GENERATION ======== #


# Standard public/private + double key generation
def create_keys(p, q, g):
    sk_a = get_random_bytes(16) # double key
    sk = secrets.randbelow(q) # secret key
    pk = pow(g, sk, p) # public key
    return pk, sk, sk_a


# ======== ANAMORPHIC ENCRYPTION ======== #


def aEncrypt(p, q, g, m, m_a, pk, sk_a, IV, st):
    # Generate pseudo-random r' to compute r
    r_prime = F(sk_a, IV, st, q)
    r = (r_prime + m_a) % q

    # Calculate ElGamal ciphertext
    pk_r = pow(pk, r, p)
    ct0 = (m * pk_r) % p
    ct1 = pow(g, r, p)
    ct = (ct0, ct1)

    # Increment state for next message
    st += 1

    return ct, st


def baby_step_giant_step(p, g, h, bound):
    if h == 1:
        return 0  # g^0 == 1
    
    # Limit bound to at least 1
    if bound <= 1:
        return None

    # Block size n ≈ sqrt(bound) for BSGS
    n = math.isqrt(bound)
    if n * n < bound:
        n += 1

    # Baby-steps: store g^j -> for all j in [0, n-1]
    baby = {}
    current = 1
    for j in range(n):
        # If collision, keep the smallest j
        if current not in baby:
            baby[current] = j
        current = (current * g) % p

    # Compute g^n
    g_n = pow(g, n, p)
    # Modular inverse of g_n modulo p
    g_n_inv = pow(g_n, p - 2, p)

    # Giant-steps: look for i, j s.t. x = i * n + j solves g^x = h
    gamma = h
    max_i = (bound + n - 1) // n  # Compute how many giant steps we need

    for i in range(max_i):
        # Check if current gamma matches any baby table value 
        if gamma in baby:
            j = baby[gamma]     # Retrieve corresponding baby-step exponent j
            x = i * n + j       # Reconstruct the full exponent x = i*n + j
            if x < bound:       # Only accept x if it is within the allowed bound
                return x
            else:
                return None
        # Move to next giant step -> multiply gamma by g^-n
        gamma = (gamma * g_n_inv) % p

    return None


# Anamorphic decryption
def aDecrypt(p, g, sk_a, ct, IV, st, q, bound):
    r_prime = F(sk_a, IV, st, q)

    # compute inverse of g^r_prime
    g_rprime = pow(g, r_prime, p)
    g_rprime_inv = pow(g_rprime, p - 2, p)
    g_m_a = (ct[1] * g_rprime_inv) % p

    m_a = baby_step_giant_step(p, g, g_m_a, bound)
    return m_a


# Standard decryption
def Decrypt(p, ct, sk):
    s = pow(ct[1], sk, p) # s = g^r^x = y^r
    s_inverse = pow(s, -1, p)
    cover_msg = (ct[0] * s_inverse) % p
    return cover_msg


# ===== Parameter initialization ===== #

p = 179769313486231590770839156793787453197860296048756011706444423684197180216158519368947833795864925541502180565485980503646440548199239100050792877003355816639229553136239076508735759914822574862575007425302077447712589550957937778424442426617334727629299387668709205606050270810842907692932019128194467627007 
q = 89884656743115795385419578396893726598930148024378005853222211842098590108079259684473916897932462770751090282742990251823220274099619550025396438501677908319614776568119538254367879957411287431287503712651038723856294775478968889212221213308667363814649693834354602803025135405421453846466009564097233813503
g = g_generator(p)

pk, sk, sk_a = create_keys(p, q, g)

st = 0
IV = 0

bound = 2**36

In [21]:
# ===== Safe prime generator ===== #

# https://www.rfc-editor.org/rfc/rfc3526#page-3
# 2048-bit safe prime from RFC 3526 (group 14)
p_hex = (
    "FFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD1"
    "29024E088A67CC74020BBEA63B139B22514A08798E3404DD"
    "EF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245"
    "E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7ED"
    "EE386BFB5A899FA5AE9F24117C4B1FE649286651ECE65381"
    "FFFFFFFFFFFFFFFF"
)

p = int(p_hex, 16)
q = (p - 1) // 2

print("p:", p)
print("q:", q)

p: 179769313486231590770839156793787453197860296048756011706444423684197180216158519368947833795864925541502180565485980503646440548199239100050792877003355816639229553136239076508735759914822574862575007425302077447712589550957937778424442426617334727629299387668709205606050270810842907692932019128194467627007
q: 89884656743115795385419578396893726598930148024378005853222211842098590108079259684473916897932462770751090282742990251823220274099619550025396438501677908319614776568119538254367879957411287431287503712651038723856294775478968889212221213308667363814649693834354602803025135405421453846466009564097233813503


In [ ]:
# ===== Int encryption test ===== #

m = 13000
m_a = 200000005
print(f'm = {m}, m_a = {m_a}')

ct, st_new = aEncrypt(p, q, g, m, m_a, pk, sk_a, IV, st)
print(f"Ciphertext: {ct}")


# ===== Int decryption test ===== #

cover_m = Decrypt(p, ct, sk)
secret_m = aDecrypt(p, g, sk_a, ct, IV, st, q, bound)
print(f'cover_m = {cover_m}, secret_m = {secret_m}')

m = 13000, m_a = 200000005
Ciphertext: (1913861343883343165279614105683747021945195540403198509746410286484321452452719416964902110752770525867769768353015993528226601430045082181132350274213853, 16004849847820936426070952803722593183942085963746635600526831786933875850736605476908309196631562944155237871097296556031948759547397099209990429891344214)
cover_m = 13000, secret_m = 200000005


In [22]:
# ===== Message encryption test ===== #

def encode_msg(m: str, m_a: str):
    # Convert each word to integers
    int_array = [encode_msg_to_int(word) for word in m.split()]
    a_int_array = [encode_msg_to_int(word_a) for word_a in m_a.split()]

    pairs = zip_longest(int_array, a_int_array, fillvalue=0)

    cts = []
    st_local = st
    for m_int, m_a_int in pairs:
        ct, st_local = aEncrypt(p, q, g, m_int, m_a_int, pk, sk_a, IV, st_local)
        cts.append(ct)

    return cts, st_local

def decode_msg(cts, st_start):
    decoded_cover = ""
    decoded_anam = ""
    st_local = st_start

    for ct in cts:
        m = Decrypt(p, ct, sk)
        m_a = aDecrypt(p, g, sk_a, ct, IV, st_local, q, bound)
        decoded_cover += decode_msg_to_int(m) + " "
        decoded_anam += (decode_msg_to_int(m_a) if m_a is not None else "[?]") + " "
        st_local += 1

    return decoded_cover.strip(), decoded_anam.strip()


m = "Hi, how are you doing James?"
m_a = "We attack at dawn, my friend" # Only len(words) <= 6 works

# Encode to ciphertext
cts, st_after = encode_msg(m, m_a)
print("Ciphertext: ", cts)

# Decode to text
decoded_m, decoded_m_a = decode_msg(cts, st)
print(f'cover_m: {decoded_m} \nsecret_m: {decoded_m_a}')

Ciphertext:  [(7371660072089335122266216865164537100117144750870744569102420217916749213936412544687516738716837167480143538952162614104643890159743454963049854314562583342821943905330939071119229227467198104291143483217264977313689425258106412153461820401190819336438091085233533916593114868269716235827533552931362937235, 103230928331917864547611151101674685645976421343239093615398343318510885226814037727393183030684647997140086457307627742811485072125583628035147707197166546652173580499361625120897694612655479559467927408160696830627753266127179849229551641470492340364972844492577263793152718660395074681058191821202213996048), (77465091351534848714491951876003703152676416073192656803765132678494035719925852717910809477463142047374120020094811751990243412458175467072260423038683827447209476570735286894105772434461289882880535823049426802100839739664128943410097860230797273270739785488412781127560497173894871297915824944787724378333, 37780295649116701392648897328460927052268498045509372

### To implement:
* ElGamal ✅
* Baby step / giant step ✅
* Implement static strong prime ✅
* Encrypt messages instead of integers ✅
* Write down progress + knowledge in Overleaf ✅
* Read research paper 🛑
* Add new ElGamal version to PM 🛑
* Implement password protection + panic password 🛑
* Final UI + ai chatbot 🛑
* Write full Overleaf paper 🛑